# Import Data

In [1]:
import pandas as pd

train_df = pd.read_csv('../../input/feedback-prize-effectiveness/train.csv')
train_df['label'] = train_df['discourse_effectiveness'].map({'Ineffective':0,'Adequate':1,'Effective':2})
print(train_df.shape)
train_df.head()

(36765, 6)


,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,label
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,1
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,1
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,1
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,1
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,1


# Ensemble

In [2]:
oof_df = train_df.copy()
oof_df['oof_ineffective_cat'] = pd.read_csv('./result/oof_cat.csv')['oof_ineffective'].values
oof_df['oof_adequate_cat'] = pd.read_csv('./result/oof_cat.csv')['oof_adequate'].values
oof_df['oof_effective_cat'] = pd.read_csv('./result/oof_cat.csv')['oof_effective'].values

oof_df['oof_ineffective_lgb'] = pd.read_csv('./result/oof_lgb.csv')['oof_ineffective'].values
oof_df['oof_adequate_lgb'] = pd.read_csv('./result/oof_lgb.csv')['oof_adequate'].values
oof_df['oof_effective_lgb'] = pd.read_csv('./result/oof_lgb.csv')['oof_effective'].values

oof_df['oof_ineffective'] = 0.5 * oof_df['oof_ineffective_cat'] + 0.5 * oof_df['oof_ineffective_lgb']
oof_df['oof_adequate']    = 0.5 * oof_df['oof_adequate_cat']    + 0.5 * oof_df['oof_adequate_lgb']
oof_df['oof_effective']   = 0.5 * oof_df['oof_effective_cat']   + 0.5 * oof_df['oof_effective_lgb']

In [3]:
from sklearn.metrics import log_loss

cols = ['oof_ineffective','oof_adequate','oof_effective']

oof_score = log_loss(oof_df['label'].values, oof_df[cols].values, labels=[0,1,2])
print('oof={:.4f}'.format(oof_score))

oof=0.5805
